In [1]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

# Pandas
import pandas as pd

import sqlite3

In [2]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

# Use `declarative_base` from SQLAlchemy to model the tables as ORM classes
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

# hawaii.sqlite is empty at this point.

In [3]:
# Create a class for `Measurement` and for `Station`. The __repr__ method tells Python how to print objects of a class
# More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr

if not 'Measurement' in dir():
    class Measurement(Base):
        __tablename__ = 'measurements'
        id = Column(Integer, primary_key=True)
        station = Column(Text)
        date = Column(Text)
        prcp = Column(Float)
        tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

if not 'Station' in dir():
    class Station(Base):
        __tablename__ = 'stations'
        station = Column(Text, primary_key=True)
        name = Column(Text)
        latitude = Column(Float)
        longitude = Column(Float)
        elavation = Column(Float)
        
        def __repr__(self):
            return f"id={self.id}, name={self.name}"

# Create the tables in the database using `create_all`. 
Base.metadata.create_all(engine)

#Load the cleaned csv file into a pandas dataframe
clean_measurements_df = pd.read_csv('clean_hawaii_measurements.csv')
clean_stations_df = pd.read_csv('clean_hawaii_stations.csv')

# set df column names as without this step, somehow extra column gets added with 'none' values in Out[8]:
clean_stations_df.columns = ['station', 'name', 'latitude', 'longitude', 'elavation']

# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
measurements_data = clean_measurements_df.to_dict(orient='records')
stations_data = clean_stations_df.to_dict(orient='records')

print(measurements_data[:5])
print(stations_data[:5])


[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]
[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elavation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elavation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elavation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elavation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'e

In [4]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [5]:
# Save the reference to the tables as variables called `measurements` and 'stations'
measurements = sqlalchemy.Table('measurements', metadata, autoload=True)
stations = sqlalchemy.Table('stations', metadata, autoload=True)

In [6]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurements.delete())
conn.execute(stations.delete())

In [7]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurements.insert(), measurements_data)
conn.execute(stations.insert(), stations_data)

In [8]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 10").fetchall()
conn.execute("select * from stations limit 9").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]